In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import logging, os,re
import time, dash
from dash import Dash
import plotly.express as px
import plotly.graph_objects as go

In [2]:
raw = pd.read_csv("../DATA/who_mortality_pays.csv", 
                  engine='python',
                  header=None, 
                  dtype=str,
                  sep=';')
header_idx = raw[0].str.startswith("Region Code").idxmax()

df = pd.read_csv("../DATA/who_mortality_pays.csv",
                 engine="python",
                 header=None,
                 dtype=str,
                 sep=';',
                 skiprows=header_idx)
# forcer le split sur les virgules
df = df[0].str.split(",", expand=True)
df.columns = [
    "Region_Code",
    "Region_Name",
    "Country_Code",
    "Country_Name",
    "Year",
    "Sex",
    "Age_group_code",
    "Age_group",
    "Number",
    "Percent_cause_specific_deaths",
    "Age_standardized_rate_100000",
    "Death_rate_per_100000",
    '',
    ""
]
df = df.drop(index=0).reset_index(drop=True)

df.head()


,Region_Code,Region_Name,Country_Code,Country_Name,Year,Sex,Age_group_code,Age_group,Number,Percent_cause_specific_deaths,Age_standardized_rate_100000,Death_rate_per_100000,,
0,EU,Europe,ALB,Albania,1987,All,Age_all,[All],72.00000000,0.41556043,3.66448761,2.34062612,,None
1,EU,Europe,ALB,Albania,1987,All,Age00,[0],0.00000000,0.00000000,,0.00000000,,None
2,EU,Europe,ALB,Albania,1987,All,Age01_04,[1-4],1.00000000,0.08097166,,0.34965035,,None
3,EU,Europe,ALB,Albania,1987,All,Age05_09,[5-9],0.00000000,0.00000000,,0.00000000,,None
4,EU,Europe,ALB,Albania,1987,All,Age10_14,[10-14],0.00000000,0.00000000,,0.00000000,,None


In [3]:
df.dtypes

Region_Code                      object
Region_Name                      object
Country_Code                     object
Country_Name                     object
Year                             object
Sex                              object
Age_group_code                   object
Age_group                        object
Number                           object
Percent_cause_specific_deaths    object
Age_standardized_rate_100000     object
Death_rate_per_100000            object
                                 object
                                 object
dtype: object

In [4]:
num_cols = ["Number", "Age_group_code", "Age_group", "Percent_cause_specific_deaths", "Age_standardized_rate_100000", "Death_rate_per_100000"]

for col in num_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")
    
df.head()

,Region_Code,Region_Name,Country_Code,Country_Name,Year,Sex,Age_group_code,Age_group,Number,Percent_cause_specific_deaths,Age_standardized_rate_100000,Death_rate_per_100000,,
0,EU,Europe,ALB,Albania,1987,All,NaN,NaN,72.0,0.415560,3.664488,2.340626,,None
1,EU,Europe,ALB,Albania,1987,All,NaN,NaN,0.0,0.000000,NaN,0.000000,,None
2,EU,Europe,ALB,Albania,1987,All,NaN,NaN,1.0,0.080972,NaN,0.349650,,None
3,EU,Europe,ALB,Albania,1987,All,NaN,NaN,0.0,0.000000,NaN,0.000000,,None
4,EU,Europe,ALB,Albania,1987,All,NaN,NaN,0.0,0.000000,NaN,0.000000,,None


In [14]:
total_deces = df.groupby(['Country_Name', "Year"])['Death_rate_per_100000'].sum().reset_index()
total_deces

,Country_Name,Year,Death_rate_per_100000
0,"""China","Hong Kong SAR""",329.383475
1,Albania,1987,926.567809
2,Albania,1988,1019.546665
3,Albania,1989,1223.842730
4,Albania,1992,900.163743
...,...,...,...
4934,Venezuela (Bolivarian Republic of),2012,3274.498260
4935,Venezuela (Bolivarian Republic of),2013,3165.451329
4936,Venezuela (Bolivarian Republic of),2014,3309.558385
4937,Venezuela (Bolivarian Republic of),2015,3332.194766


In [9]:
df['Country_Name'].values

array(['Albania', 'Albania', 'Albania', ..., 'Brazil', 'Brazil', 'Brazil'],
      shape=(210229,), dtype=object)